In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
#Scrapes the table of top 250 Beers
url = "https://www.beeradvocate.com/beer/top-rated/"
data = requests.get(url)
page = BeautifulSoup(data.text)

#Gets all elements in table
table = page.find("table")
rows = list()
#Puts all the links in a list
links = []
names = []
for link in table.find_all("a",href=True):
    links.append(link['href'])
    names.append(link.get_text())
    
#Creates Dataframe with all the needed links
links = pd.DataFrame(links)
names = pd.DataFrame(names)

df_names = pd.DataFrame(names.values.reshape(250,3))
df_final = pd.DataFrame(links.values.reshape(250,3))

df_final.columns = ["Beer","Company","Beer Type"]
df_final.insert(loc=0,column="Name",value=df_names[0])
df_final

,Name,Beer,Company,Beer Type
0,Kentucky Brunch Brand Stout,/beer/profile/23222/78820/,/beer/profile/23222/,/beer/top-styles/157/
1,Vanilla Bean Assassin,/beer/profile/23222/113831/,/beer/profile/23222/,/beer/top-styles/157/
2,Marshmallow Handjee,/beer/profile/26/42349/,/beer/profile/26/,/beer/top-styles/84/
3,Abraxas - Barrel-Aged,/beer/profile/25888/87246/,/beer/profile/25888/,/beer/top-styles/157/
4,Heady Topper,/beer/profile/46317/16814/,/beer/profile/46317/,/beer/top-styles/140/
...,...,...,...,...
245,Kentuckley,/beer/profile/34258/237135/,/beer/profile/34258/,/beer/top-styles/157/
246,Schaarbeekse Kriek,/beer/profile/2216/17647/,/beer/profile/2216/,/beer/top-styles/10/
247,Red Eye November,/beer/profile/33824/201187/,/beer/profile/33824/,/beer/top-styles/84/
248,Flora,/beer/profile/22511/62290/,/beer/profile/22511/,/beer/top-styles/129/


In [3]:
Beers_dataframe = pd.DataFrame(columns = ['Beer_Name', 'Rating', 'Review'])
names = df_final["Name"]

#Cleans HTML Tags
CLEANR = re.compile('<.*?>') 
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

counter = -1
for j in df_final['Beer']:
    #iterates through all links
    counter +=1
    url = "https://www.beeradvocate.com" + str(j)
    
    #Grabs each new url and
    beer_page = requests.get(url)
    beer_html = beer_page.text
    soup = BeautifulSoup(beer_html, 'html.parser')
    
    #finds the reviews
    reviews = soup.find_all(id= 'rating_fullview_content_2')
    for rev in reviews:
        #adds name of beer
        productname = names[counter]

        #adds user rating
        rating = str(rev.find(class_='BAscore_norm'))
        rating = cleanhtml(rating)

        #adds user text
        reviewtext = str(rev.find('div'))
        reviewtext = cleanhtml(reviewtext)

        #creates review and appends to dataframe
        entire_review = [{'Beer_Name':productname, 'Rating':rating,'Review':reviewtext}]
        Beers_dataframe = Beers_dataframe.append(entire_review)
#drops most where reviews are blank
Beers_dataframe = Beers_dataframe.drop_duplicates()

C:\Users\Sypox\AppData\Local\Temp\ipykernel_28224\1235508686.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Beers_dataframe = Beers_dataframe.append(entire_review)
C:\Users\Sypox\AppData\Local\Temp\ipykernel_28224\1235508686.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Beers_dataframe = Beers_dataframe.append(entire_review)
C:\Users\Sypox\AppData\Local\Temp\ipykernel_28224\1235508686.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Beers_dataframe = Beers_dataframe.append(entire_review)
C:\Users\Sypox\AppData\Local\Temp\ipykernel_28224\1235508686.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Beers_dataframe = Be

In [4]:
#Beers_dataframe.to_csv("Beer Reviews.csv")

In [5]:
Beers_dataframe

,Beer_Name,Rating,Review
0,Kentucky Brunch Brand Stout,None,None
0,Kentucky Brunch Brand Stout,5,I didnt think i was going to give it a perfect...
0,Kentucky Brunch Brand Stout,3.79,So I just read a review that called the legend...
0,Kentucky Brunch Brand Stout,4.64,"2021 vintage, bottle #79\n\nHoly. Fucking. Shi..."
0,Kentucky Brunch Brand Stout,4.27,"Celebrating my buddy @Rug with his 1,000th bee..."
...,...,...,...
0,Coolship Red,4.5,"Very interesting and complex ale. First, this ..."
0,Coolship Red,4.49,Appearance - The beer was pinkish red is color...
0,Coolship Red,4.5,"Look: beautiful, opaque garnet red. Soft white..."
0,Coolship Red,4.24,"Caged and corked bottle dated dec3.,2020 and o..."
